### Importing

In [1]:
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision.models import resnet50
from PIL import Image
import matplotlib.pyplot as plt

### Display Image Function

In [2]:
# Function to display images
def show_image(img, title="Image", cmap=None):
    plt.imshow(img, cmap=cmap)
    plt.title(title)
    plt.axis('off')
    plt.show()

### Feature Extraction using ResNet

In [3]:
def feature_extraction(image_path):
    # Load a pretrained ResNet model
    model = resnet50(pretrained=True)
    model.eval()

    # Define a transform to preprocess the input image
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    # Load and preprocess the image
    image = Image.open(image_path)
    input_tensor = preprocess(image)
    input_batch = input_tensor.unsqueeze(0)  # Create a mini-batch as expected by the model

    # Forward pass through the model
    with torch.no_grad():
        features = model(input_batch)

    # Extract features from the last fully connected layer
    feature_vector = features.squeeze().numpy()
    print("Feature vector shape:", feature_vector.shape)
    return image

### Edge Detection using Sobel Operator

In [4]:
def edge_detection(image_path):
    # Load and preprocess the image
    image = Image.open(image_path).convert('L')  # Convert to grayscale
    preprocess = transforms.ToTensor()
    input_tensor = preprocess(image).unsqueeze(0)  # Add batch dimension

    # Define the Sobel operator kernels
    sobel_x = torch.tensor([[-1., 0., 1.],
                            [-2., 0., 2.],
                            [-1., 0., 1.]]).unsqueeze(0).unsqueeze(0)
    sobel_y = torch.tensor([[-1., -2., -1.],
                            [0., 0., 0.],
                            [1., 2., 1.]]).unsqueeze(0).unsqueeze(0)

    # Apply Sobel operators
    grad_x = F.conv2d(input_tensor, sobel_x, padding=1)
    grad_y = F.conv2d(input_tensor, sobel_y, padding=1)

    # Calculate the gradient magnitude
    grad_magnitude = torch.sqrt(grad_x**2 + grad_y**2).squeeze().numpy()

    return grad_magnitude